# Predection :

## libraries

In [1]:
# data manipulation libraries
import pandas as pd
import numpy as np
import os
import random
from random import seed
import tarfile
seed(1121)

In [2]:
# machine learning methods
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import  SGDClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import f1_score,balanced_accuracy_score,accuracy_score
from sklearn.metrics import balanced_accuracy_score

In [ ]:
from keras.utils import to_categorical
from keras.preprocessing import sequence

In [5]:
from pprint import pprint

## Load and Preprocessing Data :

load data :

In [ ]:
#Extract the dtd from tar file
tf = tarfile.open('lab 1/data.tar')
tf.extractall()

In [9]:
train_data_path = 'lab 1/data/train'
test_data_path = 'lab 1/data/test'

In [10]:
#get the ids of the patients from the folders :
tr_patients = [p for p in sorted(os.listdir(train_data_path))]
ts_patients = [p for p in sorted(os.listdir(test_data_path))]

print('num training patients:', len(tr_patients))
print('num test patients:', len(ts_patients))

num training patients: 20000
num test patients: 10000


In [11]:
def concat_patients(dirr ,patients):
    toconcat = []
    for p in patients:
        df = pd.read_csv(dirr + '/' + p, sep = "|")
        e=df.shape[0] 
        patient_id = [p[8:-4]]* e
        df["patient_id"] = patient_id
        toconcat.append(df)
    dff = pd.concat(toconcat)
    return dff 

In [12]:
train_df = concat_patients(train_data_path, tr_patients)
test_df = concat_patients(test_data_path, ts_patients)

Data Preprocessing

In [13]:
features_train=train_df.drop(['SepsisLabel','patient_id'],axis=1)
features_test=test_df.drop(['SepsisLabel','patient_id'],axis=1)

In [14]:
#  drop columns as mentioned at the feature engineering part :
features_train = features_train[['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'Glucose', 'Age', 'Gender', 'HospAdmTime', 'EtCO2']]
features_test = features_test[['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'Glucose', 'Age', 'Gender', 'HospAdmTime', 'EtCO2']]

In [15]:
# here complet the missing data using fillna and the mean of the data

features_train=features_train.fillna(features_train.mean())
features_test=features_test.fillna(features_test.mean())


In [16]:
featurearr_train=features_train.iloc[:,:].values
featurearr_test=features_test.iloc[:,:].values


In [17]:
label_train=train_df[['SepsisLabel',"patient_id"]]
label_test=test_df[['SepsisLabel',"patient_id"]]


In [18]:
y_train_arr=label_train.iloc[:].values
y_test_arr=label_test.iloc[:].values


In [19]:
patientid_train=train_df['patient_id'].values.tolist()
patientid_test=test_df['patient_id'].values.tolist()

In [20]:
# Appending Hourly data of each patent to single row:

def append_data(patientid_t,featurearr_t,yarr_t):
    c=0
    ip=[]
    yp=[]
    seq=[]
    index=0
    for i in range(len(patientid_t)):
        if i==0:
            seq+=featurearr_t[index].tolist()
            index+=1
            continue
        if patientid_t[i]!=patientid_t[i-1]:
            ip.append(seq)
            yp+=[yarr_t[i-1,0]]
            seq=[]
        if yarr_t[i,0]==0:
            seq+=featurearr_t[index].tolist()
        elif yarr_t[i,0]==1 and patientid_t[i-1]!=patientid_t[i]:
            seq+=featurearr_t[index].tolist()
        index+=1
    ip.append(seq)
    yp+=[yarr_t[i-1,0]]
    return ip,yp

In [21]:
xp_train, yp_train = append_data(patientid_train,featurearr_train,y_train_arr)

In [22]:
xp_test, yp_test = append_data(patientid_test,featurearr_test,y_test_arr)

In [23]:
ids_test = []
for i in range(len(patientid_test)):
    if patientid_test[i]!=patientid_test[i-1]:
        ids_test.append(patientid_test[i])    

In [24]:
# get the number of the Patients that have Sepsis in the train set 
Sepsis_train=0
for i in yp_train:
    if i==1:
        Sepsis_train+=1
print('Number of Patients Diagnosed With Sepsis in the train set is',Sepsis_train) 

Number of Patients Diagnosed With Sepsis in the train set is 1415


In [25]:
# get the number of the Patients that have Sepsis in the test set 
Sepsis_test=0
for i in yp_test:
    if i==1:
        Sepsis_test+=1
print('Number of Patients Diagnosed With Sepsis in the test set is',Sepsis_test) 

Number of Patients Diagnosed With Sepsis in the test set is 741


In [ ]:
x_train=np.array(xp_train)
y_train=np.array(yp_train)

In [ ]:
x_test=np.array(xp_test)
y_test=np.array(yp_test)

In [29]:
x_train = sequence.pad_sequences(x_train, maxlen=4032)
x_test = sequence.pad_sequences(x_test, maxlen=4032)


# Training/Classification Models

## RandomForestClassifier  :

### hyperparameter of RandomForestClassifier:

In [30]:
rf = RandomForestClassifier()

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
print(rf.get_params())

Parameters currently in use:

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [31]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [100, 575, 1050, 1525, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 35, 60, 85, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [32]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()

In [35]:
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, verbose=2,cv = 3, random_state=42, n_jobs = -1)

In [ ]:
# Fit the random search model
rf_random.fit(x_train, y_train)

### Base Model :

In [37]:
rf_base = RandomForestClassifier()
rf_base.fit(x_train, y_train)
y_RF1= rf_base.predict(x_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_RF1))
print('F1 Score: %.3f' % f1_score(y_test, y_RF1))
print('Balanced Accuracy Score: %.3f' % balanced_accuracy_score(y_test, y_RF1))

Accuracy: 0.962
F1 Score: 0.693
Balanced Accuracy Score: 0.786


### Best Model :

In [38]:
rf_random.best_params_

{'n_estimators': 575,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': None,
 'bootstrap': True}

In [49]:
rf_best = RandomForestClassifier(n_estimators= 575,min_samples_split= 2,min_samples_leaf= 2,max_features= 'auto',max_depth = None, bootstrap =  True)

rf_best.fit(x_train, y_train)
y_RF2= rf_best.predict(x_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_RF2))
print('F1 Score: %.3f' % f1_score(y_test, y_RF2))
print('Balanced Accuracy Score: %.3f' % balanced_accuracy_score(y_test, y_RF2))

Accuracy: 0.962
F1 Score: 0.691
Balanced Accuracy Score: 0.785


## MLPClassifier :

### hyperparameter of MLPClassifier:

In [ ]:
mlp = MLPClassifier()

# Look at parameters used by our current MLPClassifier
print('Parameters currently in use:\n')
print(mlp.get_params())

Parameters currently in use:

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}


In [ ]:
#The ith element represents the number of neurons in the ith hidden layer:
hidden_layer_sizes =  [(10,30,10),(20,),(50,50,50,50) ,(100,)]
#Activation function for the hidden layer:
activation = ['tanh', 'relu']
#The solver for weight optimization:
solver = ['sgd', 'adam']
#Strength of the L2 regularization term:
alpha = [0.0001, 0.05]
#Learning rate schedule for weight updates:
learning_rate = ['constant','adaptive']

# Create the random grid
MLP_random_grid = {'hidden_layer_sizes': hidden_layer_sizes,
                   'activation': activation,
                   'solver': solver,
                   'alpha': alpha,
                   'learning_rate': learning_rate}

print(MLP_random_grid)

{'hidden_layer_sizes': [(10, 30, 10), (20,), (50, 50, 50, 50), (100,)], 'activation': ['tanh', 'relu'], 'solver': ['sgd', 'adam'], 'alpha': [0.0001, 0.05], 'learning_rate': ['constant', 'adaptive']}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
mlp = MLPClassifier()
# MLPClassifier of parameters, using 3 fold cross validation,
# search across 10 different combinations, and use all available cores
mlp_random = RandomizedSearchCV(estimator = mlp, param_distributions = MLP_random_grid, n_iter = 10, verbose=2,cv = 3, random_state=42, n_jobs = -1)

In [43]:
mlp_random.fit(x_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 10.2min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                           batch_size='auto', beta_1=0.9,
                                           beta_2=0.999, early_stopping=False,
                                           epsilon=1e-08,
                                           hidden_layer_sizes=(100,),
                                           learning_rate='constant',
                                           learning_rate_init=0.001,
                                           max_fun=15000, max_iter=200,
                                           momentum=0.9, n_iter_no_change=10,
                                           nesterovs_momentum=True, power_t=0.5,
                                           random...
                                           verbose=False, warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'ac

### Bese Model:

In [51]:
MLP_base = MLPClassifier()
MLP_base.fit(x_train, y_train)
y_MLP1 = MLP_base.predict(x_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_MLP1))
print('F1 Score: %.3f' % f1_score(y_test, y_MLP1))
print('Balanced Accuracy Score: %.3f' % balanced_accuracy_score(y_test, y_MLP1))

Accuracy: 0.961
F1 Score: 0.675
Balanced Accuracy Score: 0.773


### Best Model :

In [52]:
mlp_random.best_params_

{'solver': 'adam',
 'learning_rate': 'constant',
 'hidden_layer_sizes': (100,),
 'alpha': 0.05,
 'activation': 'relu'}

In [53]:
MLP_best = MLPClassifier(solver= 'adam',
 learning_rate= 'constant',
 hidden_layer_sizes= (100,),
 alpha= 0.05,
 activation= 'relu')

MLP_best.fit(x_train, y_train)
y_MLP2= MLP_best.predict(x_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_MLP2))
print('F1 Score: %.3f' % f1_score(y_test,y_MLP2))
print('Balanced Accuracy Score: %.3f' % balanced_accuracy_score(y_test, y_MLP2))

Accuracy: 0.961
F1 Score: 0.685
Balanced Accuracy Score: 0.784


##  Different models we tried : 

### Stochastic Gradient Descent 

In [47]:
sgdc = SGDClassifier(loss='hinge',max_iter=2000,tol=0.001)
sgdc.fit(x_train,y_train)
sgdc_preds = sgdc.predict(x_test)
print('Accuracy: %.3f' % accuracy_score(y_test, sgdc_preds))
print('F1 Score: %.3f' % f1_score(y_test, sgdc_preds))
print('Balanced Accuracy Score: %.3f' % balanced_accuracy_score(y_test,sgdc_preds))

Accuracy: 0.955
F1 Score: 0.583
Balanced Accuracy Score: 0.712


### Decision Tree Classifier

In [50]:
dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)
dt_preds = dt.predict(x_test)
print('Accuracy: %.3f' % accuracy_score(y_test, dt_preds))
print('F1 Score: %.3f' % f1_score(y_test, dt_preds))
print('Balanced Accuracy Score: %.3f' % balanced_accuracy_score(y_test, dt_preds))

Accuracy: 0.880
F1 Score: 0.420
Balanced Accuracy Score: 0.745


### CREAT CSV file

In [54]:
list_dict = {'Ids':ids_test, 'SepsisLabel':y_MLP2} 
df = pd.DataFrame(list_dict) 
df.to_csv('prediction.csv', index=False) 

### take the best model for prediction

In [55]:
from joblib import dump

In [56]:
dump(rf_best, 'model.joblib') 


['model.joblib']